In [7]:
import pandas as pd

Wp_per_m2 = 1000
# Conversion constants for poly-Si
kg_per_cell = 0.015  # poly-Si to wafer, ITRPV 2020
# cell_active_area = 0.0244  # m^2
cell_active_area = 0.024566  # m^2
efficiency = 0.215
wp_per_cell = cell_active_area*efficiency*Wp_per_m2
cell_to_module_ratio = 0.98
cells_per_module = 60

cells_per_kg = 1 / kg_per_cell
kg_per_module = kg_per_cell * cells_per_module
wp_per_module = wp_per_cell* cells_per_module*cell_to_module_ratio

units = ['kg', 'wafer', 'cell', 'module', 'Wp']
conversion_matrix = pd.DataFrame(1.0, index=units, columns=units)

conversions = {
    ('kg', 'cell'): kg_per_cell,
    ('wafer', 'cell'): 1,
    ('kg', 'wafer'): kg_per_cell,
    ('module', 'cell'): 1 / cells_per_module,
    ('module', 'wafer'): 1 / cells_per_module,
    ('kg', 'module'): kg_per_module,
    ('Wp', 'cell'): wp_per_cell,
    ('Wp', 'wafer'): wp_per_cell,
    ('Wp', 'module'): wp_per_module,
    ('Wp', 'kg'): wp_per_cell / kg_per_cell,
}

for (from_unit, to_unit), factor in conversions.items():
    conversion_matrix.at[from_unit, to_unit] = factor
    conversion_matrix.at[to_unit, from_unit] = 1 / factor

def convert_dataframe_units(df, target_unit, conversion_matrix=conversion_matrix):
    df_converted = df.copy()

    def convert_row(row):
        from_unit = row['Unit']
        if from_unit == target_unit:
            return row
        factor = conversion_matrix.at[from_unit, target_unit]
        row['Process Cost'] *= factor
        low, high = row['Process Cost Range']
        row['Process Cost Range'] = (low * factor, high * factor)
        row['Unit'] = target_unit
        return row

    return df_converted.apply(convert_row, axis=1)

conversion_matrix

,kg,wafer,cell,module,Wp
kg,1.000000,0.015000,0.015000,0.900000,0.002840
wafer,66.666667,1.000000,1.000000,60.000000,0.189333
cell,66.666667,1.000000,1.000000,60.000000,0.189333
module,1.111111,0.016667,0.016667,1.000000,0.003220
Wp,352.112667,5.281690,5.281690,310.563372,1.000000


5.28169